In [14]:
#TENSORFLOW IMPLEMENTATION

%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color
import cv2


SF = 2   #scaling factor, i.e. how much do you want to scale your image (here, twice of original image)

ip_height = 160
ip_width = 240

In [15]:
#there are total of 100 images (i chose 50 for training and 20 are for testing)
#but i am not using all of them 

temp1 = io.imread_collection('C:/Users/adity/Desktop/sai/*_SRF_2_LR.png')
temp2 = io.imread_collection('C:/Users/adity/Desktop/sai/*_SRF_2_HR.png')

x_ = np.array([images for i,images in enumerate(temp1)],dtype =np.float32).reshape(-1,160,240,1)
y_ = np.array([images for i,images in enumerate(temp2)],dtype= np.float32).reshape(-1,1,320*480)

xx = np.array(x_).astype(np.float32)   #x is tensor - (160, 240,1)
yy = np.array(y_).astype(np.float32)   #y is a vector- (1,320*480)
'''

change these numbers according to your
train and test set sizes....

'''
xtest =np.array(x_[:1])        #test set
ytest =np.array(y_[:1])

xx=xx[1:9]                        #train set
yy=yy[1:9]


X = tf.placeholder(tf.float32, shape=(None, 160, 240, 1),name="X" )        # input
Y = tf.placeholder(tf.float32,shape=(None, 1, 320*480),name="Y" )      # actual output
print(len(xtest))

1


In [16]:
def conv2d(x, W, b, strides=1, act_fn = 1):
    # Conv2D wrapper, with bias and relu activation(only if act_fn=1)
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    if act_fn ==1:
        return tf.nn.relu(x)
    else:
        return x

def dil_conv2d(x, W, b, rate=2):
    # Dilation Conv2D wrapper, with bias but no activation
    x = tf.nn.atrous_conv2d(x, W , rate, padding='SAME')
    x = tf.nn.bias_add(x, b)
    return x
    


In [23]:
# Store layer's weight & bias   ------   weights here are equivalent to 'filters' in tensorflow

with tf.device("/gpu:0"):
    def CNN(X):
        weights = {
            # 5x5 conv, 1 input, 32 outputs                             FEATURE REP.
            'wc1': tf.Variable(tf.random_normal([3, 3, 1, 32])*0.0008,name='wc1',dtype=tf.float32),

            # 5x5 conv, 32 inputs, 32 outputs                           SHRINKING
            'wc2': tf.Variable(tf.random_normal([3, 3, 32, 16])*0.0008,'wc2',dtype=tf.float32),

            # 5x5 conv, 32 inputs, 16 outputs                           NON-LINEAR MAPPING
            'wc3': tf.Variable(tf.random_normal([3, 3, 16, 32])*0.0008, 'wc3',dtype=tf.float32),
            # 3X3 dilation conv, 16 inputs, 16 outputs                  NON-LINEAR MAPPING
            'wd1': tf.Variable(tf.random_normal([3, 3, 32,32])*0.0008,'wd1',dtype=tf.float32),
            # 5x5 conv, 32 inputs, 16 outputs                           NON-LINEAR MAPPING
            'wc4': tf.Variable(tf.random_normal([3, 3, 32,32])*0.0008,'wc4',dtype=tf.float32),
            # 3X3 dilation conv, 16 inputs, 16 outputs                  NON-LINEAR MAPPING
            'wd2': tf.Variable(tf.random_normal([3, 3, 32,32])*0.0008,'wd2',dtype=tf.float32),
            # 5x5 conv, 32 inputs, 16 outputs                           NON-LINEAR MAPPING
            'wc5': tf.Variable(tf.random_normal([3, 3, 32,16])*0.0008,'wc5',dtype=tf.float32),

            # 5x5 conv, 16 inputs, 64 outputs                           EXPANSION
            'wc6': tf.Variable(tf.random_normal([3, 3, 16, 32])*0.0008,'wc6',dtype=tf.float32),

        }

        biases = {
            'bc1': tf.Variable(tf.random_normal([32]),dtype=tf.float32),    #FEATURE REPRESENTATION

            'bc2': tf.Variable(tf.random_normal([16]),dtype=tf.float32),     #SHRINKING

            'bc3': tf.Variable(tf.random_normal([32]),dtype=tf.float32),     #NON-LINEAR MAPPING
            'bd1': tf.Variable(tf.random_normal([32]),dtype=tf.float32),     #NON-LINEAR MAPPING
            'bc4': tf.Variable(tf.random_normal([32]),dtype=tf.float32),     #NON-LINEAR MAPPING
            'bd2': tf.Variable(tf.random_normal([32]),dtype=tf.float32),     #NON-LINEAR MAPPING
            'bc5': tf.Variable(tf.random_normal([16]),dtype=tf.float32),     #NON-LINEAR MAPPING

            'bc6': tf.Variable(tf.random_normal([32]),dtype=tf.float32),    #EXPANSION
        }

        X = tf.cast(tf.reshape(X, shape=[-1, 160, 240, 1]),tf.float32)

        c1 = conv2d(X, weights['wc1'], biases['bc1'])    #FEATURE REP

        c2 = conv2d(c1, weights['wc2'], biases['bc2'])          #SHRINKING

        c3 = conv2d(c2, weights['wc3'], biases['bc3'])          #NON-LINEAR MAPPING
        d1 = dil_conv2d(c3, weights['wd1'], biases['bd1'])
        c4 = conv2d(d1, weights['wc4'], biases['bc4'])
        d2 = dil_conv2d(c4, weights['wd2'], biases['bd2'])
        c5 = conv2d(d2, weights['wc5'], biases['bc5'])

        c6 = conv2d(c5, weights['wc6'], biases['bc6'], act_fn =0)     #EXPANSION (LOCAL QUEUE JUMPING)
        c6 = tf.add(c1,c6)
        c6 = tf.nn.relu(c6)
                                                                #DECONVOLUTION
        op_image =tf.contrib.layers.conv2d_transpose(c6, num_outputs=1, kernel_size=(3,3), stride=2, padding="same")
        op_image = tf.reshape(op_image, ( -1,1, 320*480),"op_image")     #reshape the o/p image to a vector

        return op_image

In [ ]:
with tf.device("/gpu:0"):
    def train_cnn(X):
        prediction = CNN(X)
        print(prediction)
        #mse
        loss = tf.reduce_mean(tf.pow(tf.subtract(prediction, Y), 2.0))
        #optimizer
        train1 = tf.train.AdamOptimizer(0.005).minimize(loss)
        train2 = tf.train.AdamOptimizer(0.0009).minimize(loss)
        train3 = tf.train.AdamOptimizer(0.0005).minimize(loss)
        # i used same optimizers with different learning rate.
        
        
        with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())

            for epoch in range(20):
                l = sess.run(loss, feed_dict={X: xx, Y: yy})
                sess.run(train1, feed_dict={X: xx, Y: yy})
                print('Epoch', epoch+1, 'completed,  loss:\t', l)
                
            for epoch in range(20,45):
                l = sess.run(loss, feed_dict={X: xx, Y: yy})
                sess.run(train2, feed_dict={X: xx, Y: yy})
                print('Epoch', epoch+1, 'completed,  loss:\t', l)
                
            for epoch in range(45,70):
                l = sess.run(loss, feed_dict={X: xx, Y: yy})
                sess.run(train3, feed_dict={X: xx, Y: yy})
                print('Epoch', epoch+1, 'completed,  loss:\t', l)
                
            print("\n TEST ERROR : ", sess.run(loss, feed_dict={X: xtest, Y: ytest}))
            
            a = sess.run(prediction, feed_dict={X: xtest} )
            b = np.array(a,dtype = np.uint8).reshape(1,80,120)
            
            for i in range(1):
                #save images in directory
                str = 'C:/Users/adity/Desktop/op_img_{0}_SRF_2_HR'.format(i)+'.png'
                cv2.imwrite(str,b[i])
                #show images
                fig = plt.figure()
                plt.imshow(b[i], cmap ='gray')
                

In [ ]:
train_cnn(X)

Tensor("op_image_3:0", shape=(?, 1, 153600), dtype=float32)
Epoch 1 completed,  loss:	 16574.5
Epoch 2 completed,  loss:	 15528.3
Epoch 3 completed,  loss:	 14228.9
Epoch 4 completed,  loss:	 11941.6
Epoch 5 completed,  loss:	 6737.02
Epoch 6 completed,  loss:	 85436.7
Epoch 7 completed,  loss:	 4886.78
Epoch 8 completed,  loss:	 8674.23
Epoch 9 completed,  loss:	 10453.2
Epoch 10 completed,  loss:	 11340.4
Epoch 11 completed,  loss:	 11707.2
Epoch 12 completed,  loss:	 11874.1
Epoch 13 completed,  loss:	 11942.5
Epoch 14 completed,  loss:	 11959.5
Epoch 15 completed,  loss:	 11917.3
